### Подготовка: Установка базы данных `northwind`

Если у вас еще нет этой базы данных, вы можете легко установить ее.

1.  Создайте пустую базу данных: `createdb northwind`
2.  Скачайте SQL-скрипт для создания схемы и данных (например, [отсюда](https://www.google.com/search?q=https://raw.githubusercontent.com/pthom/northwind_psql/master/northwind.sql)).
3.  Выполните скрипт с помощью `psql`:
    ```bash
    psql -d northwind -f /path/to/northwind.sql
    ```

-----

### Задание 1: Основы `\copy`, `COPY` (Экспорт и Импорт)

Эта задача проверяет ваше умение работать с форматами и опциями `\copy`, `COPY`.

1.  **Экспорт:** Выгрузите данные из таблицы `products` в файл `products_export.csv`.

      * **Требования к формату:**
          * Формат: **CSV**.
          * Разделитель полей: точка с запятой (`;`).
          * Включите заголовки столбцов (`HEADER`).
          * Значения `NULL` в столбце `reorder_level` должны быть представлены в файле как строка `(not set)`.
      * **Требования к данным:**
          * Выгрузите все столбцы, **кроме** `discontinued`.

2.  **Импорт:**

      * Создайте точную копию структуры таблицы `products`, назовите ее `products_copy`.
        ```sql
        CREATE TABLE products_copy (LIKE products INCLUDING ALL);
        ```
      * Загрузите данные из созданного вами файла `products_export.csv` в новую таблицу `products_copy`.
      * **Проверка:** Убедитесь, что количество строк в `products` и `products_copy` совпадает.

-----

### Задание 2: Продвинутый экспорт с использованием запроса

Эта задача требует объединения данных из нескольких таблиц для создания отчета.

  * **Задача:** Создать отчет о продажах в файле `sales_report.csv`.
  * **Требования:**
    1.  Напишите **один** SQL-запрос, который извлекает следующие поля:
          * `orders.order_id`
          * `orders.order_date`
          * `products.product_name`
          * `order_details.quantity`
          * `order_details.unit_price`
    2.  Используйте команду `\copy` или `COPY`, чтобы результат этого запроса сохранить в `sales_report.csv` с заголовками.
    3.  Отчет должен содержать только заказы, сделанные в **1997 году**.

-----

### Задание 3: Импорт с обработкой ошибок (Staging Table) 

Это ключевое задание, имитирующее реальную рабочую ситуацию с "грязными" данными.

Представьте, что вам прислали файл `new_suppliers.csv` с данными о новых поставщиках, который нужно загрузить в таблицу `suppliers`.

  * **Содержимое файла `new_suppliers.csv` (создайте его вручную):**

    ```csv
    supplier_name,contact_name,city,country,phone
    "Global Parts Inc.","John Smith","New York","USA","(123) 456-7890"
    "Euro Imports","","Paris","France",
    "Asia Goods Co.","Li Wei","Shanghai","China","+86 21 5555 8888"
    "Invalid Data","Bad Contact","Nowhere","??","000-000"
    ```

  * **Ваша задача:** Загрузить только валидные данные, используя **стратегию промежуточной таблицы (staging table)**.

    1.  **Создайте временную таблицу** `suppliers_staging` со всеми столбцами типа `TEXT`.
    2.  **Загрузите** содержимое `new_suppliers.csv` в `suppliers_staging` с помощью `\copy` или `COPY`. Заголовки в файле отсутствуют.
    3.  **Напишите `INSERT ... SELECT` запрос**, который переносит данные из `suppliers_staging` в основную таблицу `suppliers`.
          * **Логика очистки:**
              * Поле `supplier_id` должно генерироваться автоматически (предполагаем, что это `SERIAL`).
              * Пустые строки в `contact_name` и `phone` должны стать `NULL`.
              * Страна `"USA"` должна быть преобразована в `"USA"`.
              * Строка `"Invalid Data"` не должна попасть в целевую таблицу (например, отфильтруйте по стране `"???"`).
    4.  **Проверка:** Выполните `SELECT` из таблицы `suppliers`, чтобы убедиться, что добавились только 3 новые, корректно отформатированные записи.

-----

### Задание 4: Работа с `pg_dump` и `pg_restore` 

Это задание проверяет навыки резервного копирования и восстановления.

1.  **Создание бэкапа:**

      * Сделайте полный бэкап вашей базы данных `northwind`.
      * Используйте **custom format (`-Fc`)**.
      * Назовите файл дампа `northwind.dump`.

2.  **Выборочное восстановление:**

      * Создайте **новую, пустую** базу данных с именем `northwind_partial_restore`.
      * Используя утилиту `pg_restore` и созданный ранее файл `northwind.dump`, восстановите в `northwind_partial_restore` **только две таблицы**: `employees` и `customers`.
      * **Проверка:** Подключитесь к базе `northwind_partial_restore` и убедитесь, что в ней существуют только таблицы `employees` и `customers` (и их зависимости, если `pg_restore` их подтянул), а других таблиц (например, `products`) нет.

-----

### Задание 5: Оптимизация производительности (Теоретический вопрос) 🚀

Эта задача проверяет ваше понимание "почему" за техниками оптимизации.

  * **Вопрос:** Вам необходимо загрузить **50 миллионов** записей в таблицу `order_details` из большого CSV-файла. Таблица `order_details` имеет первичный ключ, два внешних ключа (на `orders` и `products`) и один дополнительный индекс по полю `quantity`.

    Опишите **подробный, пошаговый план действий** для выполнения этой задачи с максимальной скоростью. Для каждого шага (например, "удаление индекса") кратко **объясните, почему** этот шаг ускоряет процесс импорта.

-----